In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.nonparametric.kde import KDEUnivariate
from statsmodels.nonparametric import smoothers_lowess
from pandas import Series, DataFrame
from patsy import dmatrices
from sklearn import datasets, svm

In [3]:
df = pd.read_csv("Budget_Data.csv")

In [4]:
df.dtypes

Date            object
Account         object
Description     object
Category        object
Notes          float64
Amount         float64
Class           object
dtype: object

In [5]:
df['Amount'] = df['Amount'].abs() 

In [6]:
# Convert that column into a datetime datatype
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df.head()

,Date,Account,Description,Category,Notes,Amount,Class
0,2017-05-23,Per Chk 0001,Pending Ues-seacoast Ut Bill Ach Entry Memo Po...,ELECTRIC,NaN,56.88,DEBIT
1,2017-05-22,Per Chk 0001,Bill Payment Check Xx0003 Payee: St. Jean's Cr...,CHECK,NaN,200.00,DEBIT
2,2017-05-22,Per Chk 0001,Lowe's,HOUSE,NaN,1.84,DEBIT
3,2017-05-22,Credit Card - Ending in 7898,Tender Crop Farm,GROCERY,NaN,9.97,DEBIT
4,2017-05-22,Credit Card - Ending in 7898,Tst* Las Olas Exeter,DINNING,NaN,14.74,DEBIT


In [30]:
table = pd.pivot_table(df,index=["Category","Date"],values=["Amount"], aggfunc=np.sum)
table

Amount
Category      Date               
401K          2017-01-31    15.81
              2017-02-28    13.81
              2017-03-10    51.84
              2017-03-23    19.11
              2017-03-31    16.20
              2017-04-07   113.37
              2017-04-28     8.98
ALCOHOL       2017-01-03    16.00
              2017-01-06    19.99
              2017-01-07    19.99
              2017-02-26    11.99
              2017-03-26     9.99
              2017-04-09    48.96
              2017-04-10    20.48
              2017-04-17    11.99
              2017-04-24    25.48
              2017-04-25    24.98
              2017-04-28    40.97
              2017-04-30    22.29
              2017-05-03    10.99
              2017-05-14    14.99
ATM           2017-01-09   400.00
              2017-01-10   400.00
              2017-01-11   100.00
              2017-01-13  1155.65
              2017-01-17   100.00
              2017-01-30   550.00
              2017-01-31  1249.16
              2017-02-01   900.00
              2017-02-15  1055.65
...                           ...
PAYMENT       2017-04-01   421.48
              2017-04-02  1946.92
              2017-04-03  1000.00
              2017-04-04   521.48
              2017-04-07    97.00
              2017-04-10  1000.00
              2017-04-13   502.41
              2017-04-14   502.41
              2017-04-19  2679.36
              2017-04-25  1500.00
              2017-05-01  1946.92
              2017-05-02  1225.00
              2017-05-03  1600.00
              2017-05-09    50.00
              2017-05-12  1867.46
              2017-05-15  1867.46
              2017-05-19  2679.36
PHONE         2017-03-10   162.84
UNCATEGORIZED 2017-01-23    10.00
              2017-04-13     2.50
              2017-04-24     1.85
              2017-04-28     5.00
              2017-04-30     7.50
              2017-05-01     7.50
              2017-05-13    61.25
              2017-05-15    79.49
              2017-05-16   153.61
              2017-05-18    45.00
              2017-05-19    44.24
              2017-05-22    89.19

[520 rows x 1 columns]

In [31]:
table.query('Category == ["ALCOHOL","GROCERY","DINNING","GAS","HOBBY","HOUSE","UNCATEGORIZED","CLOTHES","COFFEE"]')

Amount
Category      Date              
ALCOHOL       2017-01-03   16.00
              2017-01-06   19.99
              2017-01-07   19.99
              2017-02-26   11.99
              2017-03-26    9.99
              2017-04-09   48.96
              2017-04-10   20.48
              2017-04-17   11.99
              2017-04-24   25.48
              2017-04-25   24.98
              2017-04-28   40.97
              2017-04-30   22.29
              2017-05-03   10.99
              2017-05-14   14.99
CLOTHES       2017-01-05   54.99
              2017-01-20   47.99
              2017-01-26   19.96
              2017-01-27   67.98
              2017-01-30   47.99
              2017-03-06    6.00
              2017-03-27   22.77
              2017-03-29   42.99
              2017-04-17   16.47
              2017-05-01   13.96
              2017-05-17   47.20
              2017-05-18    3.00
              2017-05-21   56.38
              2017-05-22   91.58
COFFEE        2017-01-03    4.40
              2017-01-04    3.22
...                          ...
HOUSE         2017-03-28   70.00
              2017-04-02   20.61
              2017-04-09    6.48
              2017-04-12   12.94
              2017-04-17   21.95
              2017-04-18   37.87
              2017-04-23   29.96
              2017-04-24   43.93
              2017-04-25    5.99
              2017-04-26   33.96
              2017-04-27   33.96
              2017-05-12   30.59
              2017-05-15    7.98
              2017-05-16   28.40
              2017-05-17    5.96
              2017-05-18   11.97
              2017-05-20   83.76
              2017-05-22   83.76
UNCATEGORIZED 2017-01-23   10.00
              2017-04-13    2.50
              2017-04-24    1.85
              2017-04-28    5.00
              2017-04-30    7.50
              2017-05-01    7.50
              2017-05-13   61.25
              2017-05-15   79.49
              2017-05-16  153.61
              2017-05-18   45.00
              2017-05-19   44.24
              2017-05-22   89.19

[294 rows x 1 columns]

In [9]:
able.resample('M').sum()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'MultiIndex'

In [ ]:
df.query('Category == ["401K"]')

In [ ]:
#How much did we spend overall? vs. How much did we make(income)?

In [ ]:
#How much did we spend in each category?

In [ ]:
#Did we spend more than last month?

In [ ]:
#Did we spend more than budgeted? (What is budgeted for each catergory?)

In [ ]:
#Which categories do we need to spend less?

In [ ]:
#Will we have enough to pay the bills in the coming weeks, months, etc.?

In [ ]:
#Did our bills go up or down?